# Introduction

In this note book Dognella aquifer data has been discussed which is one of the target of study for acea smart water analytics competition. Lets first discuss what is aquifer?

## Aquifer
According to wikipedia, "An aquifer is an underground layer of water-bearing permeable rock, rock fractures or unconsolidated materials (gravel, sand, or silt)". The water trapped underground (underground water) can be extracted by wells. The dataset for Doganella aquifer gives data about 8 pozzos (pozzo ia an Italian word for well). Doganella is in the provice of Latina, located in Italy. Dataset also gives other parameters like amount of rainfall and temperature at two places Monteporzio and Velletri which can have some influence over the depth of water and volume of water in different pozzos. We will discuss these relations in the following sections.

In [ ]:
# loading libraries
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

In [ ]:
# importing datasets
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [ ]:
df=pd.read_csv('/kaggle/input/acea-water-prediction/Aquifer_Doganella.csv')
df

In [ ]:
# getting the various properties of dataframe columns
df.describe()

In [ ]:
df.info()

In [ ]:
#Convert Date column to datetime
df['Date']=pd.to_datetime(df['Date'])


## Variation of groundwater depth for different pozzos
Lets first chaeck the variation of grounwater depth for different pozzos with time. The level in pozzos can be replenished by rainfall but overusing of these water resources can lead to gradual decrease of water level in pozzos.

In [ ]:
df_2014=df[df['Date']>='2014-01-01']
df_2014

In [ ]:
type(df_2014['Date'])

As we can see from above table that certian values are missing. Lets see what values are not defined for different columns and so we will neglect those values and filter the data accordingly.

In [ ]:
#lets filter out the data which have NaN values
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.isnull(df)

In [ ]:
pd.set_option("display.max_rows",10, "display.max_columns", None)
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month




As we can see from above the data is absent for years before 2014 so lets trim the dataframe to get data after 2014

As we can see from the above data frame the values fror all pozzos are NaN before the index 3074 so we can neglect the data before that index. Also there is missing data for some of the index numbers after 3074. Lets create a dataset for pozzos only diplaying the data after index 3074. We will take the average over months for all variables given in the dataset.

In [ ]:
df2=df.groupby(["year", "month"]).mean()

df2

In [ ]:
dtf = (pd.date_range("2004-01-01", periods=12*17, freq="M"))

In [ ]:
df2['Date']=dtf

In [ ]:
df2['Date']=pd.to_datetime(df2['Date'])

In [ ]:

df3=df2[df2['Date']>='2014-01-01']
df3.reset_index(drop=True)

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(x=df3['Date'], y=df3['Rainfall_Monteporzio'], name="Rainfall_Monteporzio")
)

fig.add_trace(
    go.Scatter(x=df3['Date'], y=df3['Rainfall_Velletri'], name="Rainfall_Velletri")
)

As we can see from the above graphs the variation of rainfall in Velletri and Monterporzio have the same trend so we will take the average of both and see the variation of parameters with respect to this average.

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(x=df3['Date'], y=df3['Temperature_Monteporzio'], name="Temperature_Monteporzio")
)

fig.add_trace(
    go.Scatter(x=df3['Date'], y=df3['Temperature_Velletri'], name="Temperature_Velletri")
)

Similar to rainfall, the trend of temperature is similar for the two rergions. Data is missing for some date points as can be seen from the discontinuities in the above graph. So we will observe the variation of various parameters with respect to the average of temperature of these two regions.

In [ ]:
df4=df3[['Rainfall_Monteporzio', 'Rainfall_Velletri']].mean(axis=1)

In [ ]:
df3=df3.reset_index(drop=True).join(pd.DataFrame(df4).reset_index(drop=True).rename(columns={0:"average rainfall"}))
df3

In [ ]:
df5=df3[['Temperature_Monteporzio', 'Temperature_Velletri']].mean(axis=1)
df3=df3.reset_index(drop=True).join(pd.DataFrame(df5).reset_index(drop=True).rename(columns={0:"average temp."}))
df3

In [ ]:



fig=go.Figure()
for i in df3.iloc[:,2:11].columns:
    fig.add_trace(go.Scatter(x=df3['Date'], y=df3[i],
                             
                           
                             text=df3['average rainfall'],
        hovertemplate=
        "<b>avg. rainfall %{text}</b><br><br>" +
        "Date: %{x|%Y/%b}<br>" +
        "depth: %{y:}<br>" ,
        
                             
                             
                             
                             
mode='markers',
             marker=dict(size=df3['average rainfall'],sizemode='area',
                         sizeref=2.*max(df3['average rainfall'])/(20.**2),sizemin=0
                                
                                    ), name=i))
    
    
    

    fig.update_layout(
    autosize=False,
    width=1500,
    height=1000,
title="Variation of depth to groundwater for different pozzos",
xaxis_title="Date",
yaxis_title="Depth to groundwater"
        ,)
    
    
fig2 = px.bar(df3, y='average temp.', x='Date')

fig.show()
fig2.show()


Click on the legend to hid or show data points for that pozzo.

### Observations:

The size of the bubble indiates the amount of rainfall for that month.

**Pozzo1:**

From the above diagram it can be seen that, depth to grounwater for pozzo1 is shallower than the rest of the pozzos. A cycle of variation of depth to groundwater for pozzo1 can be seen from the graph. 

For year 2014 the water depth is lowest for months Apr to Jul, this period also shows the significant rainfall for this year. The average temp. was also highest for this period of time.

For 2015, the lowest depth to water are for months Apr to Jun though the avverage depth for year 2015 is more than that of 2014. This may be due to to variation in cinsumption of water from the pozzo, also the rainfall was ess in year 20156 as compared to 2014. The cycle of variation of depth was maintianed in year 2015.

In year 2016, the depth increses furthur and surprisingly the lowest depth was recorded for period Nov to Dec months. The lowering of average depth can be attributed to the less rainfall and other factors(like consumption of water and higher average temp. in 2016 as compared to previous years). Similar amount of rainfall to that of year 2015 was reported and it never reached the amount of rainfall for year 2014.

2017, the depth as lowest for months feb o may. The average depth decreses this year as compared to previous one. 

In 2018 lowest depth was found in apr which is possibly due to the good amount of rainfall in the previous months. The highest depth was found in the month of jul which is one of the hottest months as seen from the data(though the data for some of the months for year 2018 is missing). During this year the depth varied over a larger range as compared to the previous months possibly due the fact that this year had hottest summer months as compared to previous ones. 

2019, the lowest depth observed in feb and highest in aug. The variation of depth occured over large range despite the fact that rainfall was pretty good during this year. This may be due to increase in water extraction from pozzos or other hidden factors.

In 2020 lowest depth was found in jan nad highest in may. Avg. rainfall was less than the last year but the depth of groundwater at the end of the year was lower than that of previous year.



**important points** 

Depth is dependent upon the amount of rainfall recieved. High amount of rainfall can reduce the depth.Lowest depth shifted to the monts in the begining as the years proceed. Highest depth occurs around the summer months. Though an anomaly occurs in the year 2016, where lowest dpeth occurs in the months of nov and dec. Despite having good rainfall in 2019 as comapred to the other years the depth kept on increasing as the year prceeded which can be attributed to the increased consumption of water from the pozzo.




**Pozzo2**

The depth gradually decreased from 2014 to oct 2015 mainly due to the good rainfall in the past months as depicted in the above diagram. From there onwards it increased about 11 units approx. till 2019. In 2019 the depth remains constant for most part of the year and this year also had good amount of rainfall as compared to the previous ones and as a result the depth of water decreased in the coming year. In month oct and nov the water level hits the lowest among all the previous years. 


**Pozzo3**

The depth decreases till sep/oct 2015 attributed to the high intensity of rainfall in the previous months. From nov 2015 on wards the depth increases till end of 2017 mainly due to scarcity of rainfall. From there onwards the water level is increasing accompined by the good amoint of rainfall. The maximum amount of variation in depth is 10 units approx. for this pozzzo.


**Pozzo4**

Water level was rising till sep 2015 and this period also had good amount of rainfall. From there onwards the water level decreased with few showers of rainfall till jan 2019. For rest of the 2019 the water level was almost constant. In 2020 the variation was withtin 1unit, highest water level for feb and lowest for mar. The water level for nov and dec was better as compared to the previous year.



**Pozzo5**

Similar to pozzo4 the water level increases till apr2015 and then started decreasing till dec. Drastic decrease in water level occured for the year 2016, a 10 unit approx decrease. From 2017 it increased and reached peak on jan2018. From 2018 to 2019 gradual decrease in water level was observed and from 2019 to 2020 the depth remained almos constant. Note that the period between 2019 and 2020 experienced significant rainfall as compared to the previous years. 

**Pozzo6**

Water level increased till oct/sep 2015. Then it decreases mar2019. After that it rises and remained constant for the rest of the year. Note that 2019 had good amount of rainfall as compared to the pervious years. In 2020 the depth to ground water increased till may and then it decreases and remained almost constant for the months of oct and nov which also experienced good amount of rainfall. Approx. 10 units of variation was observed.


**Pozzo7**

The good amount of rainfall has resulted in raising the water level inthis pozzo. The overall depth has increased over the years with some variations whenever ther is rainfall. Significant decrease in water level occured in 2015 of about 4 units ad it remained constant till 2017. 5Units of approx change occured over the yars for this pozzo. Good amount of rainfall in 2019 have increase the water level in 2020 for this pozzo. 


**Pozzo8**

Till oct2015 the depth decreased mainly due to the good maount of rainfall in the preceeding months. From there onwards, the depth increased nov2018 where it was recorded lowest in over the years. An approx change of 9 units was observed for this pozzo. In 2019 the water level increase and remained constant till may but after that it decreased despite having good rainfall in 2019.From dec 2019 onwards the depth decreased and the average water level for year 2020 was better than the previous year.

**Pozzo9**

2014: depth varied within 2 units thoughout the year except for oct where it was lowest. Water level was deepest for jan. Summers had good rainfall that may have offset the effect of tem. on the depth.

2015: depth varied between 1 unit. Rainfall was not as good as in the previous year. Jun had lowes depth and jan had highest.

2016: Average depth increased this year. The depth remained pretty much constant except for oct where it was highest. 

2017: Surprisingly the depth recorded highest for this year. About 10 unit decrese in depth. Rainfall was similar to previous year.

2018: This year had steep decrease in depth for the starting months. Lowest depth was recorded for apr. Starting from may the depth gradually decresed and recovered a little for last month.

2019: Depth gradually decreases and reached the minimum in the month of mar. from there it decreased andreached minimum in nov. In dec it recovered by approx. 1.5 unit possibly due to high rainfall in nov. Good amont of rainfall occurs during this year similar to that of 2014.

2020: This is the best year for pozzo9. The depth to grounwater is minimum out of all years. Minimum depth occurs in may and maximum in feb. Good amount of rainfall also observed during this year. The variation of depth during this month was within 1unit approx.

The depth variation is within 1 to 2 units approx. The drastic increase in depth occurs in the year 2017. Recent years have decrease in depth of this pozzo maybe due to the incresed intensity of rainfall. The depth is gradually decreasing from mid 2018 onwards.







**important points** 

Pozzo3,5,7 and 9 have a period where the water level drastically and steeply decreased, mostly during 2017-18 for pozzo 9 and 3, 2015-17 for pozzo7 and 2016-17 for pozzo5.  Highest depth is oberved for pozzo3 and lowest for pozzo1.







In [ ]:
fig=go.Figure()
df4=df3.iloc[24:,11:19]
for i in df4.columns:
    fig.add_trace(go.Scatter(x=df3.iloc[24:]['Date'], y=df4[i],
                             
                           
                             text=df3['average rainfall'],
        hovertemplate=
        "<b>avg. rainfall %{text}</b><br><br>" +
        "Date: %{x|%Y/%b}<br>" +
        "volume: %{y:}<br>" ,
        
                             
                             
                             
                             
mode='markers',
             marker=dict(size=df3['average rainfall'],sizemode='area',
                         sizeref=2.*max(df3['average rainfall'])/(20.**2),sizemin=0
                                
                                    ), name=i))
    
    
    

    fig.update_layout(
    autosize=False,
    width=1500,
    height=1000,
title="Variation of volume for different pozzos",
xaxis_title="Date",
yaxis_title="volume of pozzo"
        ,)
    
    
fig2 = px.bar(df3, y='average temp.', x='Date')

fig.show()
fig2.show()


## Observations

#### Pozzo1

The volume for pozzo 1 has increased for the later years(2018 onwards) though there is a variation of 1000-1500 units of volume. Rainfall was higher in the  earlier years (2016-18) but the volume is lower in these years as compared to the later ones. Lowest value of 197 in oct 2017 and highest in jan2020 3425.

#### Pozzo2

Volume remained pretty much constant for the mid 2016 and then decreased till mid 2017 and rose again. The volume rose despite the fact that rain was not good as compared with the initial years. lOwest value of 2873 in sep 2017 and highest one in 2020 aug, 3879.

#### Pozzo3

Volume is decreasing till date. Never recovered to the previous values. Some local hills may be due to rain but overal decreasing trend. Highest vol in mar 2016, 5264 units and lowest in jan 2020, 1435units.

#### Pozzo4


Till jun 2016 the volume remain constant, dropped in jul but due to significant rainfall the volume rises up again. Dropped again till jun 2017 and then overall rise in volume. Highest in 2020 feb, 3897 units and lowest in sep 2019, 2903 units.


#### Pozzo 5&6

Post 2017 significant increase in volume despite having not so itense rainfall as compared to 2016-17 period. Local maxima and minima for later years 2018-2020. The local minimas occurs in the months of may and sep. For 2018 the volume units in jun were similar to that of may.


#### Pozzo 7


Overall decreasing behavior of volume with the time period. 


#### Pozzo 8

Large variations in volume especially in years 2016, 2017 and 2019. Highest volume in Feb and jun 2016 of 4191 units and lowest in sep 2016, 2921 closely followed by sep 2019, 2929 units.


#### Pozzo 9

Overall decreasing behavior of volume over the earlier years 2016-17, after that local cycles of maxima nad minima occured. Highest volume of 4294 units in feb,2016 and lowest 2800 units for dec 2020.







Please upvoat of you find the notebook helpful. Any suggestions, please do comment.